In [13]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width, empty):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = dict()
        if empty == True :
            for i in range(height):
                for j in range(width):
                    coord = []
                    if i-1 >= 0:
                        coord.append((i-1,j))
                    if j-1 >= 0:
                        coord.append((i,j-1))
                    if j+1 < width:
                        coord.append((i,j+1))
                    if i+1 < height:
                        coord.append((i+1,j))
                    self.neighbors[(i,j)] = (set(coord))
        else :
            self.neighbors = {(i,j): set() for i in range(height) for j in range (width) }
            

    def info(self):
        """
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = f"{self.height} x {self.width}\n"
        txt += str(self.neighbors)
        return txt

    def __str__(self):
        """
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt

In [24]:
laby = Maze(4, 4, empty = True)
print(laby.info())

4 x 4
{(0, 0): {(0, 1), (1, 0)}, (0, 1): {(1, 1), (0, 2), (0, 0)}, (0, 2): {(0, 1), (1, 2), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(1, 1), (2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 0), (1, 2), (2, 1)}, (1, 2): {(1, 1), (0, 2), (1, 3), (2, 2)}, (1, 3): {(2, 3), (1, 2), (0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(3, 1), (1, 1), (2, 0), (2, 2)}, (2, 2): {(2, 3), (3, 2), (1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3), (2, 2)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (2, 1), (3, 0)}, (3, 2): {(3, 1), (3, 3), (2, 2)}, (3, 3): {(2, 3), (3, 2)}}


In [25]:
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



In [26]:
laby.neighbors = {
    (0, 0): {(1, 0)},
    (0, 1): {(0, 2), (1, 1)},
    (0, 2): {(0, 1), (0, 3)},
    (0, 3): {(0, 2), (1, 3)},
    (1, 0): {(2, 0), (0, 0)},
    (1, 1): {(0, 1), (1, 2)},
    (1, 2): {(1, 1), (2, 2)},
    (1, 3): {(2, 3), (0, 3)},
    (2, 0): {(1, 0), (2, 1), (3, 0)},
    (2, 1): {(2, 0), (2, 2)},
    (2, 2): {(1, 2), (2, 1)},
    (2, 3): {(3, 3), (1, 3)},
    (3, 0): {(3, 1), (2, 0)},
    (3, 1): {(3, 2), (3, 0)},
    (3, 2): {(3, 1)},
    (3, 3): {(2, 3)}
}

print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

